#### Arya Pratama Rhama Putra (5054241026)

In [3]:

from utils import *
from logic import *
import agents
import math
from inspect import getsource
from IPython.display import HTML
from tabulate import tabulate


In [4]:

from collections import deque


class WumpusAgent:
    """
    Agen cerdas untuk Wumpus World yang menggunakan logika proposisional,
    caching untuk optimasi, dan backtracking.
    """
    def __init__(self, world, initial_pos=(1,1)):
        
        self.world = world
        self.grid_size = len(world)
        self.kb = PropKB()
        self.position = initial_pos
        self.visited = {initial_pos}
        self.path = [initial_pos]
        self.gerakan = [(0,1), (0,-1), (1,0), (-1,0)]
        self.game_over = False
        self.has_gold = False

        
        self.safe_cells = {initial_pos}

        
        self.update_kb(initial_pos[0], initial_pos[1])

    def get_percepts(self, x, y):
        """Ambil percept dari koordinat (x,y) sesuai world."""
        
        world_y = self.grid_size - y
        world_x = x - 1
        if 0 <= world_y < self.grid_size and 0 <= world_x < len(self.world[0]):
            return set(self.world[world_y][world_x])
        return set()

    def update_kb(self, x, y):
        """
        Update KB dan cache safe_cells dengan inferensi manual yang cepat.
        """
        percepts = self.get_percepts(x, y)

        
        if 'B' not in percepts and 'S' not in percepts:
            for nx, ny in self.get_adjacent_cells(x, y):
                self.safe_cells.add((nx, ny))

        
        if 'B' in percepts:
            self.kb.tell(expr(f"Breeze({x},{y})"))
            possible_pits = " | ".join([f"P({nx},{ny})" for nx, ny in self.get_adjacent_cells(x, y)])
            if possible_pits:
                self.kb.tell(expr(f"Breeze({x},{y}) ==> ({possible_pits})"))
        else:
            self.kb.tell(expr(f"~Breeze({x},{y})"))
            for nx, ny in self.get_adjacent_cells(x, y):
                self.kb.tell(expr(f"~P({nx},{ny})"))

        if 'S' in percepts:
            self.kb.tell(expr(f"Stench({x},{y})"))
            possible_wumpus = " | ".join([f"W({nx},{ny})" for nx, ny in self.get_adjacent_cells(x, y)])
            if possible_wumpus:
                self.kb.tell(expr(f"Stench({x},{y}) ==> ({possible_wumpus})"))
        else:
            self.kb.tell(expr(f"~Stench({x},{y})"))
            for nx, ny in self.get_adjacent_cells(x, y):
                self.kb.tell(expr(f"~W({nx},{ny})"))

    def get_adjacent_cells(self, x, y):
        """Mendapatkan daftar koordinat tetangga yang valid."""
        cells = []
        for dx, dy in self.gerakan:
            nx, ny = x + dx, y + dy
            if 1 <= nx <= self.grid_size and 1 <= ny <= self.grid_size:
                cells.append((nx, ny))
        return cells

    def is_safe(self, x, y):
        """Versi optimal: Cek cache dulu sebelum bertanya ke KB."""
        if (x, y) in self.safe_cells:
            return True

        pit_safe_query = expr(f"~P({x},{y})")
        wumpus_safe_query = expr(f"~W({x},{y})")

        is_pit_safe = self.kb.ask(pit_safe_query)
        is_wumpus_safe = self.kb.ask(wumpus_safe_query)

        is_truly_safe = is_pit_safe is not False and is_wumpus_safe is not False
        if is_truly_safe:
            self.safe_cells.add((x,y))

        return is_truly_safe

    def _find_path_bfs(self, start, goal_neighbor, allowed_nodes):
        """Mencari rute terpendek dari start ke goal_neighbor menggunakan BFS."""
        queue = deque([[start]])
        visited_bfs = {start}

        while queue:
            path = queue.popleft()
            node = path[-1]
            if node == goal_neighbor:
                return path
            for neighbor in self.get_adjacent_cells(node[0], node[1]):
                if neighbor in allowed_nodes and neighbor not in visited_bfs:
                    visited_bfs.add(neighbor)
                    new_path = list(path)
                    new_path.append(neighbor)
                    queue.append(new_path)
        return None

    def move(self):
        """Menentukan dan melakukan satu langkah gerakan, termasuk backtracking."""
        x, y = self.position
        
        for nx, ny in self.get_adjacent_cells(x, y):
            if (nx, ny) not in self.visited and self.is_safe(nx, ny):
                print(f"✅ Menemukan tetangga aman & baru di ({nx},{ny}). Bergerak.")
                self.position = (nx, ny)
                self.visited.add(self.position)
                self.path.append(self.position)
                self.update_kb(nx, ny)
                return True

        
        print(f"🚧 Jalan buntu di ({x},{y}). Mencari rute lain...")
        safe_unvisited = [cell for cell in self.safe_cells if cell not in self.visited]

        if not safe_unvisited:
            print("❌ Tidak ada lagi sel aman yang bisa dijelajahi.")
            return False

        goal = safe_unvisited[0]
        print(f"🎯 Tujuan baru: {goal}. Merencanakan rute...")

        path_target = None
        for neighbor in self.get_adjacent_cells(goal[0], goal[1]):
            if neighbor in self.visited:
                path_target = neighbor
                break

        if path_target is None: return False

        route = self._find_path_bfs(self.position, path_target, self.visited)
        if route and len(route) > 1:
            next_step = route[1]
            print(f"🗺️ Rute ditemukan. Langkah selanjutnya: {next_step}")
            self.position = next_step
            self.path.append(self.position)
            return True

        print("⛔ Tidak bisa menemukan rute untuk backtracking.")
        return False

    def find_gold(self):
        """Loop utama untuk menjalankan agen mencari emas."""
        print(f"Memulai pencarian gold dari {self.position}")
        max_steps = 30
        step_count = 0

        while not self.has_gold and not self.game_over and step_count < max_steps:
            step_count += 1
            x, y = self.position
            percepts = self.get_percepts(x, y)
            print(f"\n--- Langkah {step_count} ---")
            print(f"Posisi: ({x},{y}), Percepts: {percepts}")

            if 'G' in percepts:
                self.has_gold = True
                print(f"🎉 GOLD DITEMUKAN di ({x},{y})!")
                break
            if 'W' in percepts or 'P' in percepts:
                self.game_over = True
                print(f"💀 Agent mati di ({x},{y})!")
                break
            if not self.move():
                self.game_over = True
                print("🏁 Tidak ada langkah aman. Agen menyerah.")
                break

        print("\n--- Hasil Akhir ---")
        if self.has_gold:
            print("SUKSES: Agent menemukan gold!")
        else:
            print("GAGAL: Agent tidak menemukan gold atau mati.")
        print(f"Total langkah: {step_count}")
        print(f"Jejak langkah: {self.path}")
        return self.has_gold


In [11]:
import numpy as np


## Menjawab Pertanyaan yang ada di modul

#### 1. Definisikan proposisi atau aturan di dunia wumpus


Jawaban dari pertanyaan ini bisa dilihat di method bagian update_kb

```python
if 'B' in percepts:
            self.kb.tell(expr(f"Breeze({x},{y})"))
            possible_pits = " | ".join([f"P({nx},{ny})" for nx, ny in self.get_adjacent_cells(x, y)])
            if possible_pits:
                self.kb.tell(expr(f"Breeze({x},{y}) ==> ({possible_pits})"))
        else:
            self.kb.tell(expr(f"~Breeze({x},{y})"))
            for nx, ny in self.get_adjacent_cells(x, y):
                self.kb.tell(expr(f"~P({nx},{ny})"))

```
codingan diatas akan mendeteksi jika percept mendeteksi 'B' maka disekiling petak tersebut akan dicurigai ada jurang. jika tidak, maka petak tersebut akan ditandai tidak 'B' dan disekilingnya tidak ada jurang

```python
if 'S' in percepts:
            self.kb.tell(expr(f"Stench({x},{y})"))
            possible_wumpus = " | ".join([f"W({nx},{ny})" for nx, ny in self.get_adjacent_cells(x, y)])
            if possible_wumpus:
                self.kb.tell(expr(f"Stench({x},{y}) ==> ({possible_wumpus})"))
        else:
            self.kb.tell(expr(f"~Stench({x},{y})"))
            for nx, ny in self.get_adjacent_cells(x, y):
                self.kb.tell(expr(f"~W({nx},{ny})"))
```

Sama aja tapi ini versi Wumpusnya


#### 2. Susunlah rangkaian proposisi (R1–Rn) secara sistematis berdasarkan aturan Wumpus World, dimulai dari posisi awal agen di [1,1] hingga dapat dibuktikan bahwa agen berhasil mencapaiemas di [2, 4] dengan aman

Untuk menyelesaikan tugas ini, mari kita lakukan percobaan

In [5]:


world = [
    [["P"], ["B"], ["B"], ["P"], ["B"]],
    [["B"], ["G"], [], ["B"], []],
    [["P"], ["B"], [],[], []],
    [["B"], [], ["S"], ["B"], []],
    [[], ["S"], ["W"], ["P"], ["B"]]
]

agent = WumpusAgent(world, initial_pos=(1,1))

print("KB sekarang:")
print(agent.kb.clauses)
x,y = agent.position
print(f"Percepts di {agent.position}:", agent.get_percepts(x,y))

%timeit np.arange(100)**2

KB sekarang:
[~Breeze(1, 1), ~P(1, 2), ~P(2, 1), ~Stench(1, 1), ~W(1, 2), ~W(2, 1)]
Percepts di (1, 1): set()
511 ns ± 4.94 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [12]:
agent.kb.clauses

[~Breeze(1, 1),
 ~P(1, 2),
 ~P(2, 1),
 ~Stench(1, 1),
 ~W(1, 2),
 ~W(2, 1),
 Breeze(1, 2),
 (P(1, 3) | P(1, 1) | P(2, 2) | ~Breeze(1, 2)),
 ~Stench(1, 2),
 ~W(1, 3),
 ~W(1, 1),
 ~W(2, 2),
 ~Breeze(2, 1),
 ~P(2, 2),
 ~P(3, 1),
 ~P(1, 1),
 Stench(2, 1),
 (W(2, 2) | W(3, 1) | W(1, 1) | ~Stench(2, 1)),
 ~Breeze(2, 2),
 ~P(2, 3),
 ~P(2, 1),
 ~P(3, 2),
 ~P(1, 2),
 ~Stench(2, 2),
 ~W(2, 3),
 ~W(2, 1),
 ~W(3, 2),
 ~W(1, 2),
 Breeze(2, 3),
 (P(2, 4) | P(2, 2) | P(3, 3) | P(1, 3) | ~Breeze(2, 3)),
 ~Stench(2, 3),
 ~W(2, 4),
 ~W(2, 2),
 ~W(3, 3),
 ~W(1, 3)]

In [13]:
agent.move()

KeyboardInterrupt: 

[,
 ~B(1, 1), 
 ~S(1, 1), 
 (~P(1, 2) | B(1, 1)), 
 (~P(2, 1) | B(1, 1)), 
 (~W(1, 2) | S(1, 1)), 
 (~W(2, 1) | S(1, 1)), 
 ]


itu adalah klausa yang didapatkan dari agen yang berada di 1,1. terdapat klausa bahwa di titik 1,1 tidak ada breze ataupun Stench. itu menandakan di titik 1,2 dan 2,1 itu aman. oleh karena itu, ketika kita tanya agent tentang apakah ada Wumpus atau pit di 1,2 atau di 2,1, akan memunculkan False

In [13]:
agent.kb.ask(expr('W(1,2) | W(2,1)')), agent.kb.ask(expr('P(1,2) | P(2,1)'))

%timeit np.arange(100)**2

848 ns ± 16.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Agent bisa berpindah ke salah satu petak tersebut

In [14]:
agent.move()

%timeit np.arange(100)**2

✅ Menemukan tetangga aman & baru di (1,2). Bergerak.
784 ns ± 8.62 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [15]:
print(agent.kb.clauses)
x,y = agent.position
print(f"Percepts di {agent.position}:", agent.get_percepts(x,y))

%timeit np.arange(100)**2

[~Breeze(1, 1), ~P(1, 2), ~P(2, 1), ~Stench(1, 1), ~W(1, 2), ~W(2, 1), Breeze(1, 2), (P(1, 3) | P(1, 1) | P(2, 2) | ~Breeze(1, 2)), ~Stench(1, 2), ~W(1, 3), ~W(1, 1), ~W(2, 2)]
Percepts di (1, 2): {'B'}
871 ns ± 34.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Agent Berpindah ke 1,2

Safe(1, 1),
 ~B(1, 1), 
 ~S(1, 1),
(~P(1, 2) | B(1, 1)),
(~P(2, 1) | B(1, 1)),
(~W(1, 2) | S(1, 1)),
(~W(2, 1) | S(1, 1)),
Safe(1, 1),
B(1, 2), 
~S(1, 2),
(P(1, 3) | ~B(1, 2)),
(P(1, 1) | ~B(1, 2)), 
(P(2, 2) | ~B(1, 2)), 
(~W(1, 3) | S(1, 2)), 
(~W(1, 1) | S(1, 2)), 
(~W(2, 2) | S(1, 2))

Ini adalah Klausa yang didapatkan agent dari perpindahan. kita mengetahui bahwa B(1,2) itu True, klausa pada agen
(P(1, 3) | ~B(1, 2)),

(P(1, 1) | ~B(1, 2)), 

(P(2, 2) | ~B(1, 2)),

ini menghasilkan jika ~P(1,3) maka ~B(1,2) lalu begitu pula dengan klausa 2 dan 3. jika B(1,2) adalah True. maka pernyataan itu dianggap salah.

awalnya kita sudah mengecek jika 1,1 tidak ada pit. dan kita tidak punya pengetahuan akan 1,3 dan 2,2. oleh karena itu agen akan milih mundur.

In [16]:
agent.move()

%timeit np.arange(100)**2

🚧 Jalan buntu di (1,2). Mencari rute lain...
🎯 Tujuan baru: (2, 1). Merencanakan rute...
🗺️ Rute ditemukan. Langkah selanjutnya: (1, 1)
853 ns ± 18.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [17]:
print(agent.kb.clauses)
x,y = agent.position
print(f"Percepts di {agent.position}:", agent.get_percepts(x,y))

%timeit np.arange(100)**2

[~Breeze(1, 1), ~P(1, 2), ~P(2, 1), ~Stench(1, 1), ~W(1, 2), ~W(2, 1), Breeze(1, 2), (P(1, 3) | P(1, 1) | P(2, 2) | ~Breeze(1, 2)), ~Stench(1, 2), ~W(1, 3), ~W(1, 1), ~W(2, 2)]
Percepts di (1, 1): set()
859 ns ± 15.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [18]:
agent.move()

%timeit np.arange(100)**2

✅ Menemukan tetangga aman & baru di (2,1). Bergerak.
868 ns ± 16.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [19]:
print(agent.kb.clauses)
x,y = agent.position
print(f"Percepts di {agent.position}:", agent.get_percepts(x,y))

%timeit np.arange(100)**2

[~Breeze(1, 1), ~P(1, 2), ~P(2, 1), ~Stench(1, 1), ~W(1, 2), ~W(2, 1), Breeze(1, 2), (P(1, 3) | P(1, 1) | P(2, 2) | ~Breeze(1, 2)), ~Stench(1, 2), ~W(1, 3), ~W(1, 1), ~W(2, 2), ~Breeze(2, 1), ~P(2, 2), ~P(3, 1), ~P(1, 1), Stench(2, 1), (W(2, 2) | W(3, 1) | W(1, 1) | ~Stench(2, 1))]
Percepts di (2, 1): {'S'}
822 ns ± 13.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


[~Breeze(1, 1), ~P(1, 2), ~P(2, 1), ~Stench(1, 1), ~W(1, 2), ~W(2, 1), Breeze(1, 2), (P(1, 3) | P(1, 1) | P(2, 2) | ~Breeze(1, 2)), ~Stench(1, 2), ~W(1, 3), ~W(1, 1), ~W(2, 2), ~Breeze(2, 1), ~P(2, 2), ~P(3, 1), ~P(1, 1), Stench(2, 1), (W(2, 2) | W(3, 1) | W(1, 1) | ~Stench(2, 1))]

klausa tersebut yang dikumpulkan agent

kita fokus pada klausa (W(2, 2) | W(3, 1) | W(1, 1) | ~Stench(2, 1))

diketahui bahwa Stench(2, 1) adalah True

W(2,2) bisa dianggap aman karena agent punya klausa bahwa ~Stench(1, 2) dimana harusnya ada bau di 1,2

In [20]:
agent.move()

%timeit np.arange(100)**2

✅ Menemukan tetangga aman & baru di (2,2). Bergerak.
815 ns ± 13.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [21]:
print(agent.kb.clauses)
x,y = agent.position
print(f"Percepts di {agent.position}:", agent.get_percepts(x,y))

%timeit np.arange(100)**2

[~Breeze(1, 1), ~P(1, 2), ~P(2, 1), ~Stench(1, 1), ~W(1, 2), ~W(2, 1), Breeze(1, 2), (P(1, 3) | P(1, 1) | P(2, 2) | ~Breeze(1, 2)), ~Stench(1, 2), ~W(1, 3), ~W(1, 1), ~W(2, 2), ~Breeze(2, 1), ~P(2, 2), ~P(3, 1), ~P(1, 1), Stench(2, 1), (W(2, 2) | W(3, 1) | W(1, 1) | ~Stench(2, 1)), ~Breeze(2, 2), ~P(2, 3), ~P(2, 1), ~P(3, 2), ~P(1, 2), ~Stench(2, 2), ~W(2, 3), ~W(2, 1), ~W(3, 2), ~W(1, 2)]
Percepts di (2, 2): set()
808 ns ± 20.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


[~Breeze(1, 1), ~P(1, 2), ~P(2, 1), ~Stench(1, 1), ~W(1, 2), ~W(2, 1), Breeze(1, 2), (P(1, 3) | P(1, 1) | P(2, 2) | ~Breeze(1, 2)), ~Stench(1, 2), ~W(1, 3), ~W(1, 1), ~W(2, 2), ~Breeze(2, 1), ~P(2, 2), ~P(3, 1), ~P(1, 1), Stench(2, 1), (W(2, 2) | W(3, 1) | W(1, 1) | ~Stench(2, 1)), ~Breeze(2, 2), ~P(2, 3), ~P(2, 1), ~P(3, 2), ~P(1, 2), ~Stench(2, 2), ~W(2, 3), ~W(2, 1), ~W(3, 2), ~W(1, 2)]

adalah klausa yang didapat oleh agent

di sana terdapat  ~P(2, 3) dan ~W(2, 3). Artinya 2,3 aman

In [22]:
agent.move()

%timeit np.arange(100)**2

✅ Menemukan tetangga aman & baru di (2,3). Bergerak.
814 ns ± 31.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [23]:
print(agent.kb.clauses)
x,y = agent.position
print(f"Percepts di {agent.position}:", agent.get_percepts(x,y))

%timeit np.arange(100)**2

[~Breeze(1, 1), ~P(1, 2), ~P(2, 1), ~Stench(1, 1), ~W(1, 2), ~W(2, 1), Breeze(1, 2), (P(1, 3) | P(1, 1) | P(2, 2) | ~Breeze(1, 2)), ~Stench(1, 2), ~W(1, 3), ~W(1, 1), ~W(2, 2), ~Breeze(2, 1), ~P(2, 2), ~P(3, 1), ~P(1, 1), Stench(2, 1), (W(2, 2) | W(3, 1) | W(1, 1) | ~Stench(2, 1)), ~Breeze(2, 2), ~P(2, 3), ~P(2, 1), ~P(3, 2), ~P(1, 2), ~Stench(2, 2), ~W(2, 3), ~W(2, 1), ~W(3, 2), ~W(1, 2), ~Breeze(2, 3), ~P(2, 4), ~P(2, 2), ~P(3, 3), ~P(1, 3), ~Stench(2, 3), ~W(2, 4), ~W(2, 2), ~W(3, 3), ~W(1, 3)]
Percepts di (2, 3): set()
824 ns ± 18 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


[~Breeze(1, 1), ~P(1, 2), ~P(2, 1), ~Stench(1, 1), ~W(1, 2), ~W(2, 1), Breeze(1, 2), (P(1, 3) | P(1, 1) | P(2, 2) | ~Breeze(1, 2)), ~Stench(1, 2), ~W(1, 3), ~W(1, 1), ~W(2, 2), ~Breeze(2, 1), ~P(2, 2), ~P(3, 1), ~P(1, 1), Stench(2, 1), (W(2, 2) | W(3, 1) | W(1, 1) | ~Stench(2, 1)), ~Breeze(2, 2), ~P(2, 3), ~P(2, 1), ~P(3, 2), ~P(1, 2), ~Stench(2, 2), ~W(2, 3), ~W(2, 1), ~W(3, 2), ~W(1, 2), ~Breeze(2, 3), ~P(2, 4), ~P(2, 2), ~P(3, 3), ~P(1, 3), ~Stench(2, 3), ~W(2, 4), ~W(2, 2), ~W(3, 3), ~W(1, 3)]

adalah klausa yang didapat agen

disana ada ~W(2, 4) dan ~P(2, 4) artinya 2,4 aman

In [24]:
agent.move()

%timeit np.arange(100)**2

✅ Menemukan tetangga aman & baru di (2,4). Bergerak.
796 ns ± 21.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [25]:
print(agent.kb.clauses)
x,y = agent.position
print(f"Percepts di {agent.position}:", agent.get_percepts(x,y))

%timeit np.arange(100)**2

[~Breeze(1, 1), ~P(1, 2), ~P(2, 1), ~Stench(1, 1), ~W(1, 2), ~W(2, 1), Breeze(1, 2), (P(1, 3) | P(1, 1) | P(2, 2) | ~Breeze(1, 2)), ~Stench(1, 2), ~W(1, 3), ~W(1, 1), ~W(2, 2), ~Breeze(2, 1), ~P(2, 2), ~P(3, 1), ~P(1, 1), Stench(2, 1), (W(2, 2) | W(3, 1) | W(1, 1) | ~Stench(2, 1)), ~Breeze(2, 2), ~P(2, 3), ~P(2, 1), ~P(3, 2), ~P(1, 2), ~Stench(2, 2), ~W(2, 3), ~W(2, 1), ~W(3, 2), ~W(1, 2), ~Breeze(2, 3), ~P(2, 4), ~P(2, 2), ~P(3, 3), ~P(1, 3), ~Stench(2, 3), ~W(2, 4), ~W(2, 2), ~W(3, 3), ~W(1, 3), ~Breeze(2, 4), ~P(2, 5), ~P(2, 3), ~P(3, 4), ~P(1, 4), ~Stench(2, 4), ~W(2, 5), ~W(2, 3), ~W(3, 4), ~W(1, 4)]
Percepts di (2, 4): {'G'}
824 ns ± 24.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


emas telah ditemukan

#### 3. Lakukan inferensi menggunakan entailments TT-entails (untuk semua mahasiswa)

aku akan membahas mengapa agen bisa menentukan jalan berdasarkan TT-entails

klausa saat agen di 1,1

~Breeze(1, 1), ~P(1, 2), ~P(2, 1), ~Stench(1, 1), ~W(1, 2), ~W(2, 1)

In [26]:
def conjoin(clauses):
    """
    Menggabungkan sebuah list berisi kalimat-kalimat logika menjadi satu
    kalimat logika besar yang dihubungkan dengan 'AND' (&).
    
    Contoh: conjoin([A, B, C]) akan menghasilkan (A & B & C)
    """
    if not clauses:
        return None
    if len(clauses) == 1:
        return clauses[0]
    
    
    conj = clauses[0]
    
    for i in range(1, len(clauses)):
        conj = conj & clauses[i]
        
    return conj


%timeit np.arange(100)**2

810 ns ± 14.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


kita butuh function conjoin untuk membuat kumpulan klausa yang dipisah koma menjadi dipisah &

In [27]:
agent = WumpusAgent(world, initial_pos=(1,1))

print("KB sekarang:")
print(agent.kb.clauses)
x,y = agent.position
print(f"Percepts di {agent.position}:", agent.get_percepts(x,y))

%timeit np.arange(100)**2

KB sekarang:
[~Breeze(1, 1), ~P(1, 2), ~P(2, 1), ~Stench(1, 1), ~W(1, 2), ~W(2, 1)]
Percepts di (1, 1): set()
827 ns ± 20.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [28]:
list_klausa = agent.kb.clauses
kb_sebagai_kalimat = conjoin(list_klausa)

tt_entails(kb_sebagai_kalimat, expr('P(1,2) or W(1,2)'))

%timeit np.arange(100)**2

835 ns ± 25.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


karena 1,2 tidak ada pit dan wumpus, oleh karena itu agen pindah ke 1,2

In [29]:
agent.move()
agent.kb.clauses

%timeit np.arange(100)**2

✅ Menemukan tetangga aman & baru di (1,2). Bergerak.
828 ns ± 26.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [30]:
list_klausa = agent.kb.clauses
kb_sebagai_kalimat = conjoin(list_klausa)

tt_entails(kb_sebagai_kalimat, expr('P(1,3) or W(1,3)')), tt_entails(kb_sebagai_kalimat, expr('P(2,2) or W(2,2)'))

%timeit np.arange(100)**2

844 ns ± 16.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Belum bisa ditentukan apakah 1,3 dan 2,2 itu aman. jadi kita kembali ke 1,1

In [31]:
agent.move()
agent.kb.clauses

%timeit np.arange(100)**2

🚧 Jalan buntu di (1,2). Mencari rute lain...
🎯 Tujuan baru: (2, 1). Merencanakan rute...
🗺️ Rute ditemukan. Langkah selanjutnya: (1, 1)
814 ns ± 17.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [32]:
agent.move()
agent.kb.clauses

%timeit np.arange(100)**2

✅ Menemukan tetangga aman & baru di (2,1). Bergerak.
840 ns ± 17.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [33]:
list_klausa = agent.kb.clauses
kb_sebagai_kalimat = conjoin(list_klausa)

tt_entails(kb_sebagai_kalimat, expr('P(3,1) or W(3,1)')), tt_entails(kb_sebagai_kalimat, expr('P(2,2) or W(2,2)'))

%timeit np.arange(100)**2

829 ns ± 13 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


3,1 bisa jadi false karena knowledge belum lengkap, namun 2,2 pasti aman

In [34]:
agent.move()
agent.kb.clauses

%timeit np.arange(100)**2

✅ Menemukan tetangga aman & baru di (2,2). Bergerak.
850 ns ± 8.14 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [35]:
# list_klausa = agent.kb.clauses
# kb_sebagai_kalimat = conjoin(list_klausa)

# tt_entails(kb_sebagai_kalimat, expr('P(3,2) or W(3,2)')), tt_entails(kb_sebagai_kalimat, expr('P(2,3) or W(2,3)'))

Maaf tidak bisa saya lanjutkan sampai selesai. karena semakin lama agent berjalan, semakin banyak klause yang ada dan semakin lama pula running timenya. inti kerjanya seperti yang sudah saya jelaskan di atas

#### 4. Proof By Resolution dan pergerakan agent mendapatkan emas

In [36]:
world = [
    [["P"], ["B"], ["B"], ["P"], ["B"]],
    [["B"], ["G"], [], ["B"], []],
    [["P"], [], [],[], []],
    [["B"], [], ["S"], ["B"], []],
    [[], ["S"], ["W"], ["P"], ["B"]]
]

agent = WumpusAgent(world, initial_pos=(1,1))

print("KB sekarang:")
print(agent.kb.clauses)
x,y = agent.position
print(f"Percepts di {agent.position}:", agent.get_percepts(x,y))

agent.find_gold()

%timeit np.arange(100)**2

KB sekarang:
[~Breeze(1, 1), ~P(1, 2), ~P(2, 1), ~Stench(1, 1), ~W(1, 2), ~W(2, 1)]
Percepts di (1, 1): set()
Memulai pencarian gold dari (1, 1)

--- Langkah 1 ---
Posisi: (1,1), Percepts: set()
✅ Menemukan tetangga aman & baru di (1,2). Bergerak.

--- Langkah 2 ---
Posisi: (1,2), Percepts: {'B'}
🚧 Jalan buntu di (1,2). Mencari rute lain...
🎯 Tujuan baru: (2, 1). Merencanakan rute...
🗺️ Rute ditemukan. Langkah selanjutnya: (1, 1)

--- Langkah 3 ---
Posisi: (1,1), Percepts: set()
✅ Menemukan tetangga aman & baru di (2,1). Bergerak.

--- Langkah 4 ---
Posisi: (2,1), Percepts: {'S'}
✅ Menemukan tetangga aman & baru di (2,2). Bergerak.

--- Langkah 5 ---
Posisi: (2,2), Percepts: set()
✅ Menemukan tetangga aman & baru di (2,3). Bergerak.

--- Langkah 6 ---
Posisi: (2,3), Percepts: set()
✅ Menemukan tetangga aman & baru di (2,4). Bergerak.

--- Langkah 7 ---
Posisi: (2,4), Percepts: {'G'}
🎉 GOLD DITEMUKAN di (2,4)!

--- Hasil Akhir ---
SUKSES: Agent menemukan gold!
Total langkah: 7
Jejak lan

Agent berhasil menemukan emas dengan jejak langkah [(1, 1), (1, 2), (1, 1), (2, 1), (2, 2), (2, 3), (2, 4)]

In [37]:
agent.kb.clauses

%timeit np.arange(100)**2

826 ns ± 25.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [38]:
pl_resolution(agent.kb, expr('~P(2,4) & ~W(2,4)'))

%timeit np.arange(100)**2

802 ns ± 20.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Karena 2,4 gaada Pit dan Wumpus, maka 2,4 yang berisi gold dapat diraih oleh agent


Kita akan membedah secara manual

untuk membuktikan proof by resolution, kita harus menegasikan ~P(2,4) & ~W(2,4) menjadi P(2,4) | W(2,4)

lalu segala yang berhubungan tentang stench di sekirar 2,4 dan breze sekitar 2,4 dihapus. begitu terus hingga akhirnya semua klausa habis dicoret. jika klausa habis, maka resolusion terbukti